Задание 1. Датафреймы.
Цель: Получить необходимые данные из датасета.
Источник данных: https://pastebin.com/b3QGeJ8f
Описание: Требуется скачать по ссылке предложенный датасет, загрузить его в датафрейм, подготовить и получить следующие данные:
1) Список стран с максимальной и минимальной плотностью населения и их плотность
2) Число стран без береговой линии (coastline) и процент таких стран от общего числа стран
3) Регион, где меньше всего людей владеют телефонами (phones_per_1000 - количество людей, владеющих телефонами, на 1000 человек)
4) Список стран, у которых отсутствует показатель грамотности (literacy), и при этом численность населения страны (population) более 1млн человек
5) Топ 5 стран с самым низким показателем рождаемости (birthrate) по отношению к показателю смертности (deathrate)
6) Страны, в которых преобладает сельское хозяйство (agriculture) над промышленностью (industry) и сферой обслуживания (service) и при этом средний показатель миграции (net migration) по региону положительный
Результат выполнения: Jupyter Notebook с отображением результата каждого из пунктов
Основные библиотеки:
    • pandas – для работы с данными

Импортируем библиотеки

In [1]:
import pandas as pd
import requests

Скачиваем данные

In [2]:
def download(url: str, fname: str):
  r = requests.get(url, allow_redirects=True)
  with open(fname, 'wb') as f:
    f.write(r.content)

download('https://raw.githubusercontent.com/Vambat-01/test_task_magnit/main/resources/world_countries.txt', 'data.csv')

Считываем данные в DataFrame

In [3]:
data = pd.read_csv('data.csv', sep=',', decimal=",")

1) Список стран с максимальной и минимальной плотностью населения и их плотность

In [4]:
population_density = pd.DataFrame(
    {
        "country": data["country"],
        "density": data["population"] / data["area"]
    }
)

print(f"Минимальная плотность населения (топ-5):\n {population_density.sort_values(by=['density'])[:5]}")
print(f"Максимальная плотность населения (топ-5):\n {population_density.sort_values(by=['density'], ascending=False)[:5]}")

Минимальная плотность населения (топ-5):
              country   density
80        Greenland   0.026020
223  Western Sahara   1.026346
139        Mongolia   1.810751
70    French Guiana   2.192407
143         Namibia   2.476499
Максимальная плотность населения (топ-5):
         country       density
138     Monaco   16271.500000
122      Macau   16183.035714
184  Singapore    6482.178932
91   Hong Kong    6355.706960
78   Gibraltar    3989.714286


2) Число стран без береговой линии (coastline) и процент таких стран от общего числа стран

In [5]:
data["coastline"] = pd.to_numeric(data["coastline"])
not_the_coastline = len(data[data["coastline"] == 0])
print(f"Число стран без береговой линии - {not_the_coastline}")

coefficient = not_the_coastline / data["coastline"].shape[0]
percent = coefficient * 100
print(f"Процент стран без береговой линии от общего числа стран - {percent}")

Число стран без береговой линии - 44
Процент стран без береговой линии от общего числа стран - 19.383259911894275


In [ ]:
3) Регион, где меньше всего людей владеют телефонами (phones_per_1000 - количество людей, владеющих телефонами, на 1000 человек)

In [6]:
data['phones_per_1000'] = pd.to_numeric(data['phones_per_1000'], errors='coerce')
region_min_phone = data.groupby("region")['phones_per_1000'].sum().idxmin()
print(f"Регион, где меньше всего людей владеют телефонами - {region_min_phone}")

Регион, где меньше всего людей владеют телефонами - NORTHERN AFRICA                    


4) Список стран, у которых отсутствует показатель грамотности (literacy), и при этом численность населения страны (population) более 1млн человек

In [7]:
data[pd.isnull(data["literacy"])][data["population"] >= 1000000][['country', 'population']].sort_values('population')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,country,population
74,Gaza Strip,1428757
123,Macedonia,2050554
222,West Bank,2460492
25,Bosnia & Herzegovina,4498976
185,Slovakia,5439448


5) Топ 5 стран с самым низким показателем рождаемости (birthrate) по отношению к показателю смертности (deathrate)

In [8]:
new_data = pd.DataFrame(
    {
        "country": data["country"],
        "relationships": data["birthrate"] / data["deathrate"]
    }
)
res = new_data.sort_values("relationships")
res[:5]

,country,relationships
211,Ukraine,0.612926
30,Bulgaria,0.676244
114,Latvia,0.676428
169,Russia,0.679181
138,Monaco,0.711851


6) Страны, в которых преобладает сельское хозяйство (agriculture) над промышленностью (industry) и сферой обслуживания (service) и при этом средний показатель миграции (net migration) по региону положительный

In [9]:
regions = data.groupby("region").agg(net_migration=('net migration', 'sum'))
data2 = pd.DataFrame(
    {
        "country": data["country"],
        "region": data["region"],
        "predominance_agriculture_over_industry": data["agriculture"] - data["industry"],
        "predominance_agriculture_over_service": data["agriculture"] - data["service"],
    }
)
data3 = data2.join(regions, on="region")
data4 = data3[(data3["predominance_agriculture_over_industry"] > 0) & (data3["predominance_agriculture_over_service"] > 0) & (data3["net_migration"] > 0)]
data4

,country,region,predominance_agriculture_over_industry,predominance_agriculture_over_service,net_migration
32,Burma,ASIA (EX. NEAR EAST),0.482,0.211,38.43
113,Laos,ASIA (EX. NEAR EAST),0.168,0.197,38.43
